In [1]:
"""
1. 安装 YOLOX 环境 
"""
%cd /content/
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git
%cd YOLOX
# 切换到 0.3.0 版本
!git checkout -b 0.3.0 0.3.0
%pip install -U pip && pip install -r requirements.txt
%pip install -v -e . 

/content
Cloning into 'YOLOX'...
remote: Enumerating objects: 1850, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 1850 (delta 17), reused 32 (delta 6), pack-reused 1782
Receiving objects: 100% (1850/1850), 6.91 MiB | 28.52 MiB/s, done.
Resolving deltas: 100% (1076/1076), done.
/content/YOLOX
Switched to a new branch '0.3.0'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
"""
2. 下载 yolox_s 模型
"""
%cd /content/drive/MyDrive/my-object-detection/yolox-map-demo
!pip install wget

def download_models():
    """  
    下载 yolox_s 模型
    """
    import os
    import wget
    if not os.path.exists('models/yolox_s.pth'):
        os.mkdir('models')
        wget.download('https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_s.pth','models')
    print('models/yolox_s.pth')


download_models()

/content/drive/MyDrive/my-object-detection/yolox-map-demo
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=a80da920a57fba2281a9be00a86a3c8088660741609b6915be3e1db6485c4dc8
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
models/yolox_s.pth


In [ ]:
"""
3. 数据集
---
数据集: https://drive.google.com/file/d/12s_QoE9FxqNEgO6c6hgRQ_ySNonQWBza/view?usp=share_link
"""
#------------------
# 我的 Google Drive 路径，本地测试可直接下载数据集，解压到 datasets 文件夹
#------------------
%cd /content/drive/MyDrive/my-object-detection/yolox-map-demo

def datasets_download():
    """
    下载数据集 
    """
    import os 
    if not  os.path.exists('datasets'):
        os.mkdir('datasets')
    # 若使用 colab ，可直接挂载 drive ，进行复制操作；否则自行下载，并进行解压
    if not os.path.exists('datasets/shanghai-map-datasets'):
        os.system('cp /content/drive/MyDrive/my-datasets/shanghai-map-datasets.zip ./datasets/')
        # 解压，可自行进行下载，解压到 datasets 文件夹
        os.system('unzip -d datasets/map-datasets datasets/shanghai-map-datasets.zip')
        # 删除，多余文件
        os.system('rm datasets/shanghai-map-datasets.zip')

    
# 下载
datasets_download()

In [11]:
"""
4. 数据集转换 
---
将 labelimg 标注数据集，转换为 coco 数据集，使用 tools/labelimg2voc.py 
"""

%cd /content/drive/MyDrive/my-object-detection/yolox-map-demo/datasets/


#--------------------------------------------------------
# 训练图片后缀为 .png
#--------------------------------------------------------

from tools import labelimg2voc

labelimg2voc(
    labelimg_dir='map-datasets',
    image_suffix='.png'
)


/content/drive/MyDrive/my-object-detection/yolox-map-demo/datasets
VOCdevkit/VOC2007/ImageSets/Main/train.txt
VOCdevkit/VOC2007/ImageSets/Main/val.txt
VOCdevkit/VOC2007/ImageSets/Main/trainval.txt
VOCdevkit/VOC2007/ImageSets/Main/test.txt


In [16]:
"""
5. 训练数据
"""
%cd /content/drive/MyDrive/my-object-detection/yolox-map-demo/

# 训练
!python train.py -f voc_exp.py -d 1 -b 8 --fp16 -o -c models/yolox_s.pth

/content/drive/MyDrive/my-object-detection/yolox-map-demo
2023-02-22 02:23:52 | INFO     | yolox.core.trainer:130 - args: Namespace(batch_size=8, cache=None, ckpt='models/yolox_s.pth', devices=1, dist_backend='nccl', dist_url=None, exp_file='voc_exp.py', experiment_name='voc_exp', fp16=True, logger='tensorboard', machine_rank=0, name=None, num_machines=1, occupy=True, opts=[], resume=False, start_epoch=None)
2023-02-22 02:23:52 | INFO     | yolox.core.trainer:131 - exp value:
╒═══════════════════╤════════════════════════════╕
│ keys              │ values                     │
╞═══════════════════╪════════════════════════════╡
│ seed              │ None                       │
├───────────────────┼────────────────────────────┤
│ output_dir        │ './YOLOX_outputs'          │
├───────────────────┼────────────────────────────┤
│ print_interval    │ 10                         │
├───────────────────┼────────────────────────────┤
│ eval_interval     │ 10                         │
├────────

In [18]:
"""
6. 模型评估
"""
%cd /content/drive/MyDrive/my-object-detection/yolox-map-demo/
%cp YOLOX_outputs/voc_exp/best_ckpt.pth models/voc_best_ckpt.pth

# 评估命令
!python eval.py -f voc_exp.py -c models/voc_best_ckpt.pth -b 16 -d 1 --conf 0.001 --fp16 --fuse

/content/drive/MyDrive/my-object-detection/yolox-map-demo
2023-02-22 02:28:16 | INFO     | __main__:139 - Args: Namespace(batch_size=16, ckpt='models/voc_best_ckpt.pth', conf=0.001, devices=1, dist_backend='nccl', dist_url=None, exp_file='voc_exp.py', experiment_name='voc_exp', fp16=True, fuse=True, legacy=False, machine_rank=0, name=None, nms=None, num_machines=1, opts=[], seed=None, speed=False, test=False, trt=False, tsize=None)
2023-02-22 02:28:16 | INFO     | __main__:149 - Model Summary: Params: 8.94M, Gflops: 26.76
2023-02-22 02:28:16 | INFO     | __main__:150 - Model Structure:
YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
   

In [23]:
"""
7. 图片测试 
"""
%cd /content/drive/MyDrive/my-object-detection/yolox-map-demo/

# 测试
!python voc_demo.py image -f voc_exp.py -c models/voc_best_ckpt.pth --path assets/test.png --conf 0.25 --nms 0.45 --tsize 640 --save_result --device gpu

/content/drive/MyDrive/my-object-detection/yolox-map-demo
2023-02-22 02:33:05.224 | INFO     | __main__:main:262 - Args: Namespace(camid=0, ckpt='models/voc_best_ckpt.pth', conf=0.25, demo='image', device='gpu', exp_file='voc_exp.py', experiment_name='voc_exp', fp16=False, fuse=False, legacy=False, name=None, nms=0.45, path='assets/test.png', save_result=True, trt=False, tsize=640)
2023-02-22 02:33:05.545 | INFO     | __main__:main:272 - Model Summary: Params: 8.94M, Gflops: 26.76
2023-02-22 02:33:08.109 | INFO     | __main__:main:285 - loading checkpoint
2023-02-22 02:33:08.283 | INFO     | __main__:main:289 - loaded checkpoint done.
2023-02-22 02:33:10.014 | INFO     | __main__:inference:168 - Infer time: 1.4901s
2023-02-22 02:33:10.026 | INFO     | __main__:image_demo:205 - Saving detection result in ./YOLOX_outputs/voc_exp/vis_res/2023_02_22_02_33_08/test.png


In [26]:
"""
8. 删除缓存
"""
%cd /content/drive/MyDrive/my-object-detection/yolox-map-demo
# 删除世代模型结果
%rm -r YOLOX_outputs


/content/drive/MyDrive/my-object-detection/yolox-map-demo


In [27]:
"""
9. 上传代码
"""
!git config --global user.name "LABELNET"
!git config --global user.email 1406046087@qq.com 
!git add .
!git commit -m 'edit voc '
!git push origin main

[main 0e6bd39] add voc datasets train
 7 files changed, 397 insertions(+), 14 deletions(-)
 create mode 100644 assets/voc_test_result.png
 rewrite voc-map.ipynb (85%)
 create mode 100644 voc_datasets.py
Enumerating objects: 19, done.
Counting objects: 100% (19/19), done.
Delta compression using up to 2 threads
Compressing objects: 100% (11/11), done.
Writing objects: 100% (11/11), 391.69 KiB | 11.52 MiB/s, done.
Total 11 (delta 7), reused 0 (delta 0)
remote: Resolving deltas: 100% (7/7), completed with 7 local objects.
To https://github.com/LABELNET/yolox-map-demo.git
   7bce55b..0e6bd39  main -> main
